In [1]:
# Mount Google Drive (for Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q pandas numpy

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import json

In [4]:
# Load quality filtered dataset
FILTERED_PATH = '/content/drive/Othercomputers/My Laptop/HIN_SIN/dataset/quality_filtered.csv'
filtered_df = pd.read_csv(FILTERED_PATH, encoding='utf-8')
print(f"Quality filtered samples: {len(filtered_df)}")

# Load human validated sample (if available)
try:
    VALIDATED_PATH = '/content/drive/Othercomputers/My Laptop/HIN_SIN/annotations/human_validated_sample.csv'
    validated_df = pd.read_csv(VALIDATED_PATH, encoding='utf-8')
    print(f"Human validated samples: {len(validated_df)}")
    has_validation = True
except:
    print("No human validation data found. Proceeding without it.")
    has_validation = False

Quality filtered samples: 22558
Human validated samples: 998


In [5]:
# If human validation exists, identify samples to remove
samples_to_remove = set()

if has_validation:
    # Remove samples where:
    # 1. Intent was NOT preserved
    # 2. Code-mixing was NOT natural
    # 3. Both annotators disagreed with ground truth

    for idx, row in validated_df.iterrows():
        remove = False

        # Check intent preservation
        if 'Intent_Preserved' in row and row['Intent_Preserved'] == 'No':
            remove = True

        # Check code-mixing naturalness
        if 'CodeMix_Natural' in row and row['CodeMix_Natural'] == 'No':
            remove = True

        # Check if both annotators disagreed with ground truth
        if 'Label_Annotator1' in row and 'Label_Annotator2' in row:
            a1 = row['Label_Annotator1']
            a2 = row['Label_Annotator2']
            gt = row['Label']
            if a1 != gt and a2 != gt:
                remove = True

        if remove:
            samples_to_remove.add(row['ID'])

    print(f"Samples flagged for removal from validation: {len(samples_to_remove)}")

Samples flagged for removal from validation: 431


In [6]:
# Create final dataset
final_df = filtered_df[~filtered_df['ID'].isin(samples_to_remove)].copy()

print(f"\n=== Dataset Size Progression ===")
print(f"After quality filtering: {len(filtered_df)}")
print(f"Removed by validation: {len(samples_to_remove)}")
print(f"Final dataset size: {len(final_df)}")


=== Dataset Size Progression ===
After quality filtering: 22558
Removed by validation: 431
Final dataset size: 22127


In [7]:
# Prepare final dataset columns
# Standard format for the Sinhala-English cyberbullying dataset

final_columns = {
    'ID': 'id',
    'Text_SinhalaEnglish': 'text',
    'Label': 'label'
}

# Rename and select columns
if 'Text_SinhalaEnglish' in final_df.columns:
    text_col = 'Text_SinhalaEnglish'
elif 'Translated_Text' in final_df.columns:
    text_col = 'Translated_Text'
else:
    text_col = final_df.columns[final_df.columns.str.contains('Text', case=False)][0]

output_df = final_df[['ID', text_col, 'Label']].copy()
output_df.columns = ['id', 'text', 'label']

# Reset index
output_df = output_df.reset_index(drop=True)
output_df['id'] = range(1, len(output_df) + 1)

print(f"Final dataset shape: {output_df.shape}")
output_df.head(10)

Final dataset shape: (22127, 3)


,id,text,label
0,1,You're awesome!,0
1,2,Tum jaise logon se baat nahi karte.,1
2,3,Thanks for your support yaar.,0
3,4,Teri aukaat kya hai samjhta hai?,1
4,5,"Nice effort, proud of you.",0
5,6,Bakwas band kar.,1
6,7,"Good job, well done!",0
7,8,Tum jaise logon se baat nahi karte.,1
8,9,Keep it up buddy!,0
9,10,Idiot kahi ka!,1


In [8]:
# Dataset statistics
print("=" * 60)
print("FINAL DATASET STATISTICS")
print("=" * 60)

print(f"\nTotal samples: {len(output_df)}")
print(f"\nLabel distribution:")
label_counts = output_df['label'].value_counts()
print(f"  Non-bullying (0): {label_counts[0]} ({label_counts[0]/len(output_df):.1%})")
print(f"  Bullying (1): {label_counts[1]} ({label_counts[1]/len(output_df):.1%})")

# Text length statistics
output_df['text_length'] = output_df['text'].str.len()
output_df['word_count'] = output_df['text'].str.split().str.len()

print(f"\nText length (characters):")
print(f"  Mean: {output_df['text_length'].mean():.1f}")
print(f"  Min: {output_df['text_length'].min()}")
print(f"  Max: {output_df['text_length'].max()}")

print(f"\nWord count:")
print(f"  Mean: {output_df['word_count'].mean():.1f}")
print(f"  Min: {output_df['word_count'].min()}")
print(f"  Max: {output_df['word_count'].max()}")

FINAL DATASET STATISTICS

Total samples: 22127

Label distribution:
  Non-bullying (0): 8835 (39.9%)
  Bullying (1): 13292 (60.1%)

Text length (characters):
  Mean: 27.5
  Min: 14
  Max: 40

Word count:
  Mean: 5.1
  Min: 2
  Max: 8


In [9]:
# Check for Sinhala content
import re

def has_sinhala(text):
    """Check if text contains Sinhala characters."""
    sinhala_pattern = re.compile(r'[\u0D80-\u0DFF]')
    return bool(sinhala_pattern.search(str(text)))

def has_english(text):
    """Check if text contains English characters."""
    return bool(re.search(r'[a-zA-Z]', str(text)))

output_df['has_sinhala'] = output_df['text'].apply(has_sinhala)
output_df['has_english'] = output_df['text'].apply(has_english)
output_df['is_code_mixed'] = output_df['has_sinhala'] & output_df['has_english']

print("\nCode-mixing analysis:")
print(f"  Has Sinhala: {output_df['has_sinhala'].sum()} ({output_df['has_sinhala'].mean():.1%})")
print(f"  Has English: {output_df['has_english'].sum()} ({output_df['has_english'].mean():.1%})")
print(f"  Code-mixed (both): {output_df['is_code_mixed'].sum()} ({output_df['is_code_mixed'].mean():.1%})")


Code-mixing analysis:
  Has Sinhala: 0 (0.0%)
  Has English: 22127 (100.0%)
  Code-mixed (both): 0 (0.0%)


In [10]:
# Remove temporary analysis columns
output_df = output_df[['id', 'text', 'label']]

# View final samples
print("\n=== Sample Entries ===")
print("\n--- Non-bullying samples ---")
for _, row in output_df[output_df['label'] == 0].head(3).iterrows():
    print(f"  [{row['id']}] {row['text']}")

print("\n--- Bullying samples ---")
for _, row in output_df[output_df['label'] == 1].head(3).iterrows():
    print(f"  [{row['id']}] {row['text']}")


=== Sample Entries ===

--- Non-bullying samples ---
  [1] You're awesome!
  [3] Thanks for your support yaar.
  [5] Nice effort, proud of you.

--- Bullying samples ---
  [2] Tum jaise logon se baat nahi karte.
  [4] Teri aukaat kya hai samjhta hai?
  [6] Bakwas band kar.


In [11]:
# Save final dataset
OUTPUT_PATH = '/content/drive/Othercomputers/My Laptop/HIN_SIN/dataset/SinhalaEnglish_final.csv'
# Or for local: OUTPUT_PATH = '../dataset/SinhalaEnglish_final.csv'

output_df.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')
print(f"Saved final dataset to: {OUTPUT_PATH}")

Saved final dataset to: /content/drive/Othercomputers/My Laptop/HIN_SIN/dataset/SinhalaEnglish_final.csv


In [12]:
# Create dataset documentation (datacard)
datacard = f"""
# Sinhala-English Code-Mixed Cyberbullying Dataset

## Overview
This dataset contains Sinhala-English code-mixed text samples labeled for cyberbullying detection.
It was created by translating a Hindi-English code-mixed cyberbullying dataset.

## Dataset Statistics
- **Total samples**: {len(output_df)}
- **Non-bullying (0)**: {label_counts[0]} ({label_counts[0]/len(output_df):.1%})
- **Bullying (1)**: {label_counts[1]} ({label_counts[1]/len(output_df):.1%})

## Creation Pipeline
1. **Source**: Hindi-English code-mixed cyberbullying dataset
2. **Translation**: Hindi → Sinhala using IndicTrans2
3. **Code-mixing preservation**: English tokens preserved using language detection
4. **Quality filtering**: XLM-RoBERTa zero-shot classification for label consistency
5. **Human validation**: Partial manual validation with inter-annotator agreement

## File Format
- **Format**: CSV (UTF-8 encoded)
- **Columns**:
  - `id`: Unique identifier
  - `text`: Sinhala-English code-mixed text
  - `label`: Binary label (0 = non-bullying, 1 = bullying)

## Labels
- **0 (Non-bullying)**: Positive, neutral, or supportive content
- **1 (Bullying)**: Toxic, offensive, or cyberbullying content

## Usage
```python
import pandas as pd
df = pd.read_csv('SinhalaEnglish_final.csv', encoding='utf-8')
```

## Citation
[Add your citation here]

## License
[Add license information]

## Created
{datetime.now().strftime('%Y-%m-%d')}
"""

# Save datacard
DATACARD_PATH = '/content/drive/Othercomputers/My Laptop/HIN_SIN/dataset/README.md'
with open(DATACARD_PATH, 'w', encoding='utf-8') as f:
    f.write(datacard)
print(f"Saved dataset documentation to: {DATACARD_PATH}")
print(datacard)

Saved dataset documentation to: /content/drive/Othercomputers/My Laptop/HIN_SIN/dataset/README.md

# Sinhala-English Code-Mixed Cyberbullying Dataset

## Overview
This dataset contains Sinhala-English code-mixed text samples labeled for cyberbullying detection.
It was created by translating a Hindi-English code-mixed cyberbullying dataset.

## Dataset Statistics
- **Total samples**: 22127
- **Non-bullying (0)**: 8835 (39.9%)
- **Bullying (1)**: 13292 (60.1%)

## Creation Pipeline
1. **Source**: Hindi-English code-mixed cyberbullying dataset
2. **Translation**: Hindi → Sinhala using IndicTrans2
3. **Code-mixing preservation**: English tokens preserved using language detection
4. **Quality filtering**: XLM-RoBERTa zero-shot classification for label consistency
5. **Human validation**: Partial manual validation with inter-annotator agreement

## File Format
- **Format**: CSV (UTF-8 encoded)
- **Columns**:
  - `id`: Unique identifier
  - `text`: Sinhala-English code-mixed text
  - `label`:

In [16]:
# # Create metadata JSON
# metadata = {
#     "dataset_name": "Sinhala-English Code-Mixed Cyberbullying Dataset",
#     "version": "1.0",
#     "created_date": datetime.now().isoformat(),
#     "source_dataset": "Hindi-English Code-Mixed Cyberbullying Dataset",
#     "statistics": {
#         "total_samples": int(len(output_df)),
#         "non_bullying_count": int(label_counts[0]),
#         "bullying_count": int(label_counts[1]),
#         "class_balance": round(label_counts[1] / len(output_df), 3)
#     },
#     "pipeline": [
#         "IndicTrans2 translation (Hindi → Sinhala)",
#         "Code-mixing preservation",
#         "XLM-RoBERTa quality filtering",
#         "Human validation"
#     ],
#     "languages": ["Sinhala", "English"],
#     "task": "Binary classification (cyberbullying detection)"
# }

# METADATA_PATH = '/content/drive/Othercomputers/My Laptop/dataset/metadata.json'
# with open(METADATA_PATH, 'w', encoding='utf-8') as f:
#     json.dump(metadata, f, indent=2, ensure_ascii=False)
# print(f"Saved metadata to: {METADATA_PATH}")


# Create metadata JSON
import os

metadata = {
    "dataset_name": "Sinhala-English Code-Mixed Cyberbullying Dataset",
    "version": "1.0",
    "created_date": datetime.now().isoformat(),
    "source_dataset": "Hindi-English Code-Mixed Cyberbullying Dataset",
    "statistics": {
        "total_samples": int(len(output_df)),
        "non_bullying_count": int(label_counts[0]),
        "bullying_count": int(label_counts[1]),
        "class_balance": round(label_counts[1] / len(output_df), 3)
    },
    "pipeline": [
        "IndicTrans2 translation (Hindi → Sinhala)",
        "Code-mixing preservation",
        "XLM-RoBERTa quality filtering",
        "Human validation"
    ],
    "languages": ["Sinhala", "English"],
    "task": "Binary classification (cyberbullying detection)"
}

METADATA_PATH = '/content/drive/Othercomputers/My Laptop/HIN_SIN/dataset/metadata.json'

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(METADATA_PATH), exist_ok=True)

with open(METADATA_PATH, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)
print(f"Saved metadata to: {METADATA_PATH}")

Saved metadata to: /content/drive/Othercomputers/My Laptop/HIN_SIN/dataset/metadata.json


In [17]:
# Final summary
print("\n" + "=" * 60)
print("🎉 DATASET CREATION COMPLETE!")
print("=" * 60)

print(f"""
📊 Final Dataset:
   - File: SinhalaEnglish_final.csv
   - Samples: {len(output_df)}
   - Labels: 0 (non-bullying), 1 (bullying)

📁 Output Files:
   - dataset/SinhalaEnglish_final.csv (main dataset)
   - dataset/README.md (documentation)
   - dataset/metadata.json (metadata)

✅ Pipeline completed:
   1. Translation (Hindi → Sinhala)
   2. Code-mixing preservation
   3. Quality filtering
   4. Human validation
   5. Finalization

🔬 Ready for:
   - Model training
   - Research publication
   - Benchmarking
""")


🎉 DATASET CREATION COMPLETE!

📊 Final Dataset:
   - File: SinhalaEnglish_final.csv
   - Samples: 22127
   - Labels: 0 (non-bullying), 1 (bullying)

📁 Output Files:
   - dataset/SinhalaEnglish_final.csv (main dataset)
   - dataset/README.md (documentation)
   - dataset/metadata.json (metadata)

✅ Pipeline completed:
   1. Translation (Hindi → Sinhala)
   2. Code-mixing preservation
   3. Quality filtering
   4. Human validation
   5. Finalization

🔬 Ready for:
   - Model training
   - Research publication
   - Benchmarking

